In [3]:
import datetime
from KITE import *
import pandas as pd
import os
import getpass


instrument_token = 256265
from_datetime = datetime.datetime.now() - datetime.timedelta(days=25)     # From last 25 days
to_datetime = datetime.datetime.now()
interval = "day"

In [4]:
user_id = "ISE420"       # Login Id
password = getpass.getpass("Enter password: ") 
twofa = input('Enter twofa : ')
# KitePass = os.getenv("KITE_PASSWORD")
# kiteUserId = os.getenv("KITE_USER_ID")


# print(kiteUserId)

encToken = get_enctoken(user_id,password, twofa)

kiteObj = KITE(enctoken=encToken)


In [14]:
def data(instrument_token):
  start_date = datetime.datetime.now()- datetime.timedelta(days=365)
  end_date = datetime.datetime.now()
  df = pd.DataFrame(kiteObj.historical_data(instrument_token, start_date, end_date, interval="day"))
  df = df.drop('volume',axis=1)
  return df
  

In [45]:
df=data(256265)

In [20]:
import ta
import datetime

def bb(df):
    indicator_bb = ta.volatility.BollingerBands(
        df['close'], window=10, window_dev=2)
    width = indicator_bb.bollinger_wband()
    current_width = width[len(width)-1]
    prv_width = width[len(width)-2]
    pr_chng = ((current_width-prv_width)/prv_width)*100
    if pr_chng <= 0:
        return 0
    elif pr_chng <= 25:
        return 1
    elif pr_chng <= 50:
        return 2
    elif pr_chng <= 75:
        return 3
    else:
        return 4


In [23]:
def adx(df):
    df['adx'] = ta.trend.ADXIndicator(
        df['high'], df['low'], df['close'], window=14).adx()
    # df['adx'] = adx
    df['+DI'] = ta.trend.ADXIndicator(
        df['high'], df['low'], df['close'], window=14).adx_pos()
    df['-DI'] = ta.trend.ADXIndicator(
        df['high'], df['low'], df['close'], window=14).adx_neg()
    if df['adx'][len(df)-1] >= 20 and df['+DI'][len(df)-1] > df['-DI'][len(df)-1]:
        return 1
    return 0

In [28]:
print(df)

                         date      open      high       low     close
0   2023-03-06 00:00:00+05:30  17680.35  17799.95  17671.95  17711.45
1   2023-03-08 00:00:00+05:30  17665.75  17766.50  17602.25  17754.40
2   2023-03-09 00:00:00+05:30  17772.05  17772.35  17573.60  17589.60
3   2023-03-10 00:00:00+05:30  17443.80  17451.50  17324.35  17412.90
4   2023-03-13 00:00:00+05:30  17421.90  17529.90  17113.45  17154.30
..                        ...       ...       ...       ...       ...
242 2024-02-27 00:00:00+05:30  22090.20  22218.25  22085.65  22198.35
243 2024-02-28 00:00:00+05:30  22214.10  22229.15  21915.85  21951.15
244 2024-02-29 00:00:00+05:30  21935.20  22060.55  21860.65  21982.80
245 2024-03-01 00:00:00+05:30  22048.30  22353.30  22047.75  22338.75
246 2024-03-02 00:00:00+05:30  22406.95  22463.65  22366.95  22378.40

[247 rows x 5 columns]


In [44]:
def WMA(df):
  wma_13 = ta.trend.WMAIndicator(close=df['close'],window=13,fillna=True).wma()
  wma_55 = ta.trend.WMAIndicator(close=df['close'],window=13,fillna=True).wma()
  if(wma_13.iloc[-1]>wma_55.iloc[-1]):
    return 1
  else:
    return 0

In [46]:
def shortTerm(instrument_token):
  stock_data=data(instrument_token)
  return bb(stock_data)+WMA(stock_data)+adx(stock_data)

In [47]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return 'Hello, World! This is a Stock Recommender.'

@app.route('/shortterm')
def about():
    return shortTerm(256265)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
